In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:


!pip install torch torchvision matplotlib --quiet

# Importing Libraries
import os
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision.models import ResNet50_Weights

# Paths
train_dir_a = '/content/drive/MyDrive/Comys_Hackathon5/Task_A/train'
val_dir_a = '/content/drive/MyDrive/Comys_Hackathon5/Task_A/val'

# Data Augmentation Transforms
transform_a_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3),
    transforms.RandomApply([transforms.GaussianBlur(3)], p=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

transform_a_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Loaded Dataset
train_dataset_a = datasets.ImageFolder(root=train_dir_a, transform=transform_a_train)
val_dataset_a = datasets.ImageFolder(root=val_dir_a, transform=transform_a_val)

train_loader_a = DataLoader(train_dataset_a, batch_size=32, shuffle=True)
val_loader_a = DataLoader(val_dataset_a, batch_size=32, shuffle=False)

print("Class to index mapping:", train_dataset_a.class_to_idx)

# Visualising the Training Batch
def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis('off')
    plt.show()

dataiter = iter(train_loader_a)
images, labels = next(dataiter)
imshow(torchvision.utils.make_grid(images))

# Model Setup
model = models.resnet50(weights=ResNet50_Weights.DEFAULT)
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 2)
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Loss & Optimizer
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# Training with Early Stopping and Saving the Best Model
epochs = 20
patience = 4
train_losses = []
val_accuracies = []
best_acc = 0
wait = 0

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader_a:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    scheduler.step()
    train_losses.append(running_loss / len(train_loader_a))

    # Final Evaluation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader_a:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_acc = 100 * correct / total
    val_accuracies.append(val_acc)
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader_a):.4f}, Val Accuracy: {val_acc:.2f}%, Val Loss: {val_loss/len(val_loader_a):.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        wait = 0
        torch.save(model.state_dict(), '/content/best_gender_model.pth')
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping triggered")
            break

print(f"Best Validation Accuracy: {best_acc:.2f}%")

# Plot Accuracy & Loss
plt.plot(train_losses, label='Train Loss')
plt.plot(val_accuracies, label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.title('Training Loss & Validation Accuracy')
plt.legend()
plt.show()

Class to index mapping: {'female': 0, 'male': 1}
